# 3.4. Multiple types in one table - billboard

## 3.1.2. Column headers are values, not variable names - billboard

In [1]:
options(stringsAsFactors = FALSE)
library(lubridate)
library(reshape2)
library(stringr)
library(plyr)

raw <- read.csv("../../data/billboard.csv")
raw <- raw[, c("year", "artist.inverted", "track", "time", "date.entered",
               "x1st.week", "x2nd.week", "x3rd.week", "x4th.week", "x5th.week", "x6th.week", "x7th.week", "x8th.week",
               "x9th.week", "x10th.week", "x11th.week", "x12th.week", "x13th.week", "x14th.week", "x15th.week", "x16th.week",
               "x17th.week", "x18th.week", "x19th.week", "x20th.week", "x21st.week", "x22nd.week", "x23rd.week", "x24th.week",
               "x25th.week", "x26th.week", "x27th.week", "x28th.week", "x29th.week", "x30th.week", "x31st.week", "x32nd.week",
               "x33rd.week", "x34th.week", "x35th.week", "x36th.week", "x37th.week", "x38th.week", "x39th.week", "x40th.week",
               "x41st.week", "x42nd.week", "x43rd.week", "x44th.week", "x45th.week", "x46th.week", "x47th.week", "x48th.week",
               "x49th.week", "x50th.week", "x51st.week", "x52nd.week", "x53rd.week", "x54th.week", "x55th.week", "x56th.week",
               "x57th.week", "x58th.week", "x59th.week", "x60th.week", "x61st.week", "x62nd.week", "x63rd.week", "x64th.week",
               "x65th.week", "x66th.week", "x67th.week", "x68th.week", "x69th.week", "x70th.week", "x71st.week", "x72nd.week",
               "x73rd.week", "x74th.week", "x75th.week", "x76th.week")]
names(raw)[2] <- "artist"

raw$artist <- iconv(raw$artist, "MAC", "ASCII//translit")

raw$track <- str_replace(raw$track, " \\(.*?\\)", "")
long_name <- nchar(raw$track) > 20
raw$track[long_name] <- paste0(substr(raw$track[long_name], 0, 20), "...")

names(raw)[-(1:5)] <- str_c("wk", 1:76)

raw <- arrange(raw, year, artist, track)

clean <- melt(raw, id = 1:5, na.rm = T)
clean$week <- as.integer(str_replace_all(clean$variable, "[^0-9]+", ""))
clean$variable <- NULL
clean$date.entered <- ymd(clean$date.entered)
clean$date <- clean$date.entered + weeks(clean$week - 1)
clean$date.entered <- NULL
clean <- rename(clean, c("value" = "rank"))
clean <- arrange(clean, year, artist, track, time, week)
clean <- clean[c("year", "artist", "time", "track", "date", "week", "rank")]


Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date


Attaching package: 'plyr'

The following object is masked from 'package:lubridate':

    here



In [2]:
dim(clean)
head(clean)

[1] 5307    7

year,artist,time,track,date,week,rank
2000,2 Pac,4:22,Baby Don't Cry,2000-02-26,1,87
2000,2 Pac,4:22,Baby Don't Cry,2000-03-04,2,82
2000,2 Pac,4:22,Baby Don't Cry,2000-03-11,3,72
2000,2 Pac,4:22,Baby Don't Cry,2000-03-18,4,77
2000,2 Pac,4:22,Baby Don't Cry,2000-03-25,5,87
2000,2 Pac,4:22,Baby Don't Cry,2000-04-01,6,94


## 3.4. Multiple types in one table - billboard

In [3]:
# Normalisation --------------------------------------------------------------

song <- unrowname(unique(clean[c("artist", "track", "time")]))
song$id <- 1:nrow(song)


In [4]:
dim(song)
head(song)

[1] 317   4

artist,track,time,id
2 Pac,Baby Don't Cry,4:22,1
2Ge+her,The Hardest Part Of ...,3:15,2
3 Doors Down,Kryptonite,3:53,3
3 Doors Down,Loser,4:24,4
504 Boyz,Wobble Wobble,3:35,5
98?,Give Me Just One Nig...,3:24,6


In [5]:
rank <- join(clean, song, match = "first")


Joining by: artist, time, track


In [6]:
dim(rank)
head(rank)

[1] 5307    8

artist,time,track,year,date,week,rank,id
2 Pac,4:22,Baby Don't Cry,2000,2000-02-26,1,87,1
2 Pac,4:22,Baby Don't Cry,2000,2000-03-04,2,82,1
2 Pac,4:22,Baby Don't Cry,2000,2000-03-11,3,72,1
2 Pac,4:22,Baby Don't Cry,2000,2000-03-18,4,77,1
2 Pac,4:22,Baby Don't Cry,2000,2000-03-25,5,87,1
2 Pac,4:22,Baby Don't Cry,2000,2000-04-01,6,94,1


In [7]:
rank <- rank[c("id", "date", "rank")]


In [8]:
dim(rank)
head(rank)

[1] 5307    3

id,date,rank
1,2000-02-26,87
1,2000-03-04,82
1,2000-03-11,72
1,2000-03-18,77
1,2000-03-25,87
1,2000-04-01,94
